## Welcome to the BioProv tutorials!

### Tutorial index
* <a href="./introduction.ipynb">Introduction to BioProv</a>
* <a href="./w3c-prov.ipynb">W3C-PROV projects</a>

## Introduction to BioProv

BioProv is a library to record provenance information of bioinformatics workflows. If you work with genomics, you've probably encountered the situation where you have several different files for a number of biological samples, and each file concerns a certain aspect of your data. As you develop your analysis workflow, it is challenging to keep track of the **provenance** of your data: how, when and why each file was created and/or modified. There are many tools to aid in this task, such as [version control](https://journals.plos.org/ploscompbiol/article?id=10.1371/journal.pcbi.1004668), [scientific workflow management systems](https://www.researchgate.net/profile/Jonathan_Mercier/publication/331422146_Evaluating_Workflow_Management_Systems_A_Bioinformatics_Use_Case/links/5d961df9299bf1c363f577b4/Evaluating-Workflow-Management-Systems-A-Bioinformatics-Use-Case.pdf), or even simply keeping a [tidy computational notebook](https://journals.plos.org/ploscompbiol/article?id=10.1371/journal.pcbi.1004385).

Although these practices are certainly helpful and [we recommend that you employ them](http://journals.plos.org/ploscompbiol/article?id=10.1371/journal.pcbi.1005510), it is not trivial to integrate and share provenance information across different people, research groups and even computing environments. A solution to this has been the development of [W3C-PROV](https://www.w3.org/TR/prov-overview/), a standard created by the W3C organization to facilitate the exchange of provenance data in the web.

The W3C-PROV is composed of a set of [13 documents](https://www.w3.org/TR/prov-overview/), of which maybe the most pertinent to us is the [W3C-PROV-DM](https://www.w3.org/TR/2013/REC-prov-dm-20130430/), which describes a data model to represent provenance information. Although this model is widely implemented in a range of domain applications, including to [scientific workflows](http://www.vldb.org/pvldb/vol11/p2082-sousa.pdf), to the best of our knowledge, there is not yet a software tool specialized in the provenance of biological data structures and bioinformatics workflows. To extract provenance attributes of common file formats and common project organization patterns in bioinformatics, generic provenance extraction systems must be extended or customized, which can be a costly task for both the domain specialist and the developers of said systems. In order to fulfill this gap, we present BioProv, which aims to facilitate the provenance extraction in bioinformatics workflows by providing a Python library which integrates two open source libraries: [BioPython](https://academic.oup.com/bioinformatics/article/25/11/1422/330687) and [Prov](https://prov.readthedocs.io/en/latest/).

### How it works

BioProv is **project-based**, where each **Project** contains a number of **Samples** which have associated **Files**. **Files** may also be associated directly with the **Project**, if they contain information about zero or multiple samples. BioProv also stores information about **Programs** used create new and modify existing **Files**. **Programs** may contain **Parameters** which will determine how they will be run. Once a **Program** has been run, information about the process will be stored as a **Run**.

Therefore, these are the main classes of the BioProv library:
* **Project**
* **Sample**
* **Files**
* **Programs**
* **Parameters**
* **Runs**

See an example on how to make a BioProv Project.

A **Project** is composed of **Samples**. We are going to create a **Project** with only 1 **Sample**, which is going to be the genome sequence of [*Synechococcus elongatus* PCC 6301](https://www.ncbi.nlm.nih.gov/Taxonomy/Browser/wwwtax.cgi?id=269084), a cyanobacteria. We are going to add attributes to this **Sample** using the *attributes* argument, which takes a Python `dict`. We are going to fill it with information about [its location on NCBI](https://www.ncbi.nlm.nih.gov/assembly/GCF_000010065.1).

We then create a **Project** using a list of **Samples**.

In [1]:
import bioprov as bp 

sample = bp.Sample("Synechococcus_elongatus_PCC_6301",
                   attributes={"ncbi_accession": "GCF_000010065.1",
                               "ncbi_database": "assembly"}
                  )

project = bp.Project(samples=[sample,], tag="introduction")

### Adding Files and Programs

Now we have a **Project** containing 1 **Sample**. However, our sample has no associated **Files** nor **Programs**. Let's add a **File** to our **Sample** and run a program on it.

BioProv comes with an auxiliary `data` subpackage, which contains some preset data for us to experiment with. The `synechococcus_genome` variable is an instance of `pathlib.PosixPath`, which is used to hold file paths.

In [2]:
from bioprov.data import synechococcus_genome

# We create a File object based on a path or a string representing a path.
assembly_file = bp.File(synechococcus_genome, tag="assembly")

# We can add this File to our Sample
sample.add_files(assembly_file)
sample.files

{'assembly': /Users/vini/anaconda3/envs/bioprov/lib/python3.7/site-packages/bioprov/data/genomes/GCF_000010065.1_ASM1006v1_genomic.fna}

Now our instance of `Sample` holds a `File` object. Files can be accessed by the attribute `.files`, which is a dictionary composed of `{file.tag: File instance}`.

We can now run a **Program** in our **Sample**. The sample's **Files** can be used as **Parameter** to the program. Programs are processed by the UNIX shell. 

Here we are setting up a program using UNIX's `grep` to count the occurrences of a particular kmer in our sample. We are then going to write the results to a new **File**.

To write our program, we start with an instance of the **Program** class and add **Parameters** to it.

In [3]:
grep = bp.Program("grep")

Now that we have a **Program**, let's set the other variables we'll need.

The first is an easy one, the kmer we are going to count:

In [4]:
kmer = "GATTACA"

Then, we need to create a new **File** in our **Sample**. The files associated with a **Sample** can be accessed in the `Sample.files` dictionary:

In [5]:
sample.files

{'assembly': /Users/vini/anaconda3/envs/bioprov/lib/python3.7/site-packages/bioprov/data/genomes/GCF_000010065.1_ASM1006v1_genomic.fna}

We are going to create a new **File** in this dictionary. To set the path of this **File**, we can use attributes from the existing files.

Each item in the `Sample.files` dictionary is an instance of `bioprov.File`, so there are several attributes which may be useful:

In [6]:
print(sample.files['assembly'].__class__, "\n")
sample.files['assembly'].__dict__

<class 'bioprov.src.files.File'> 



{'path': PosixPath('/Users/vini/anaconda3/envs/bioprov/lib/python3.7/site-packages/bioprov/data/genomes/GCF_000010065.1_ASM1006v1_genomic.fna'),
 'name': 'GCF_000010065.1_ASM1006v1_genomic',
 'basename': 'GCF_000010065.1_ASM1006v1_genomic.fna',
 'directory': PosixPath('/Users/vini/anaconda3/envs/bioprov/lib/python3.7/site-packages/bioprov/data/genomes'),
 'extension': '.fna',
 'tag': 'assembly',
 'attributes': {},
 '_exists': True,
 '_size': '2.6 MB',
 'raw_size': 2730026,
 'sha1': 'f8658496b343257690f828ec14226644dc9e9ca2',
 '_document': None,
 '_entity': None}

For example, if we want to create the new file in the same directory as the `'assembly'` file, we can use its `File.directory` attribute:

In [7]:
newfilepath = sample.files['assembly'].directory.joinpath(f"{kmer}_count.txt")

We can now set a new file in the `Sample.files` dictionary based on our kmer and the directory of the existing `'assembly'` **File**:

In [8]:
sample.files[f'{kmer}_count'] = bp.File(newfilepath)

# which is the same as:
sample.add_files(bp.File(newfilepath, tag=f"{kmer}_count"))

Updating file GATTACA_count with value /Users/vini/anaconda3/envs/bioprov/lib/python3.7/site-packages/bioprov/data/genomes/GATTACA_count.txt.


### Running Programs

We now have a new **File** with the name of our kmer in the `Sample.files` dictionary.

Now, we must create the parameters to be added to the `grep` program. Parameters are strings which are added to the program's command-line. We can just put a string with all of our parameters, but creating them one by one and enclosing them with the `bp.Parameter` class will allow for querying later. Parameters are added to a **Program** with the `Program.add_parameter()` method. We then bind the **Program** to the **Sample** using the `Sample.add_programs()` method. It's important to remember these two methods: `Program.add_parameter()` and `Sample.add_programs()`. They allow BioProv to resolve internal relationships between each class. 

Finally, we check our command is correct: each `bioprov.Program` instance has a `Program.cmd` attribute which shows the exact command-line which will be run on the UNIX shell.

In [9]:
count = bp.Parameter("-c")
kmer_param = bp.Parameter(f"'{kmer}'")
in_file = bp.Parameter(str(sample.files['assembly']))
pipe_out = bp.Parameter(">", str(sample.files[f'{kmer}_count']))

for param in (count, kmer_param, in_file, pipe_out):
    grep.add_parameter(param)
    
sample.add_programs(grep)
    
grep.cmd

"/usr/bin/grep -c 'GATTACA' /Users/vini/anaconda3/envs/bioprov/lib/python3.7/site-packages/bioprov/data/genomes/GCF_000010065.1_ASM1006v1_genomic.fna > /Users/vini/anaconda3/envs/bioprov/lib/python3.7/site-packages/bioprov/data/genomes/GATTACA_count.txt"

Now we want to run our program. We use the `Program.run()` method. 

In [10]:
grep.run()

Running program 'grep'.
Command is:
/usr/bin/grep \ 
	-c 'GATTACA' \ 
	/Users/vini/anaconda3/envs/bioprov/lib/python3.7/site-packages/bioprov/data/genomes/GCF_000010065.1_ASM1006v1_genomic.fna > \ 
	/Users/vini/anaconda3/envs/bioprov/lib/python3.7/site-packages/bioprov/data/genomes/GATTACA_count.txt


Run of Program 'grep' with 4 parameter(s).
Started at Tue Oct 20 15:55:24 2020.
Ended at Tue Oct 20 15:55:25 2020.
Status is finished.

When we run a **Program**, we create a new **Run**. The `bioprov.Run` class holds information about a process, such as the start time and end time. Runs are stored in the `Program.runs` attribute:

In [11]:
grep.runs

{'1': Run of Program 'grep' with 4 parameter(s).
 Started at Tue Oct 20 15:55:24 2020.
 Ended at Tue Oct 20 15:55:25 2020.
 Status is finished.}

In [12]:
# Each Run has useful attributes such as stdout, stderr and status
grep.runs['1'].__dict__

{'program': Program 'grep' with 4 parameter(s).,
 'cmd': "/usr/bin/grep -c 'GATTACA' /Users/vini/anaconda3/envs/bioprov/lib/python3.7/site-packages/bioprov/data/genomes/GCF_000010065.1_ASM1006v1_genomic.fna > /Users/vini/anaconda3/envs/bioprov/lib/python3.7/site-packages/bioprov/data/genomes/GATTACA_count.txt",
 'params': OrderedDict([('-c', Parameter with command string '-c '),
              ("'GATTACA'", Parameter with command string ''GATTACA' '),
              ('/Users/vini/anaconda3/envs/bioprov/lib/python3.7/site-packages/bioprov/data/genomes/GCF_000010065.1_ASM1006v1_genomic.fna',
               Parameter with command string '/Users/vini/anaconda3/envs/bioprov/lib/python3.7/site-packages/bioprov/data/genomes/GCF_000010065.1_ASM1006v1_genomic.fna '),
              ('>',
               Parameter with command string '> /Users/vini/anaconda3/envs/bioprov/lib/python3.7/site-packages/bioprov/data/genomes/GATTACA_count.txt')]),
 'sample': None,
 'process': <subprocess.Popen at 0x7fd993

### Exporting Projects
We now have a simple, yet complete, **BioProv Project**. We have a **Project** with 1 or more associated **Samples**, and 1 or more **Programs** have been **run** on the sample. We can export this **Project** as a JSON file.

In [ ]:
project['Synechococcus_elongatus_PCC_6301'].files

In [ ]:
project.to_json("./introduction.json")

This project can be easily retrieved with the `bioprov.from_json()` function.

In [ ]:
project = bp.from_json("./introduction.json")

In [ ]:
project.to_json("introduction_export.json")

This allows us to read and write Projects as JSON files, so we can store and/or query them.

In [ ]:
!diff ./introduction.json ./introduction_export.json


### What about the provenance?

We've learned the basics of BioProv, like the main classes, how to create **Projects**, **Samples**, and **Programs.** However, the point of BioProv is to be able to convert these elements to the W3C-PROV format. You can couple BioProv Projects (or any other BioProv object, for that matter) to W3C-PROV elements, allowing them to be exported as W3C-PROV documents, implemented with the [Prov](https://github.com/trungdong/prov) library. Continue to the <a href="./w3c-prov.ipynb">W3C-PROV tutorial.</a>